<a href="https://colab.research.google.com/github/Edgal4/Edgal4/blob/main/FireDucks_benchmark_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fireducks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.6 MB/s eta 0:00:00


In [2]:
import time

In [3]:
# download the dataset:

import urllib.request
s3_path = "https://modin-datasets.s3.amazonaws.com/testing/yellow_tripdata_2015-01.csv"
urllib.request.urlretrieve(s3_path, "taxi.csv")

('taxi.csv', <http.client.HTTPMessage at 0x7e9b2448fa00>)

In [4]:
import fireducks.pandas as fpd
import pandas as pd

In [5]:
from fireducks.core import get_fireducks_options

get_fireducks_options().set_benchmark_mode(True)

### Read CSV

In [6]:
%timeit fpd.read_csv("taxi.csv")

1.06 s ± 91.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%timeit pd.read_csv("taxi.csv")

3.23 s ± 56.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## To CSV

In [8]:
df_pd = pd.read_csv("taxi.csv")
df_fd = fpd.read_csv("taxi.csv")

In [9]:
%timeit df_pd.to_csv("dataset_dummy_pandas.csv")

13.6 s ± 54.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit df_fd.to_csv("dataset_dummy_fireducks.csv")

2.27 s ± 209 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Memory Usage

In [11]:
df_fd.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933507 entries, 0 to 1933506
Data columns (total 21 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0             int64  
 1   Unnamed: 0             int64  
 2   VendorID               int64  
 3   tpep_pickup_datetime   object 
 4   tpep_dropoff_datetime  object 
 5   passenger_count        int64  
 6   trip_distance          float64
 7   RatecodeID             int64  
 8   store_and_fwd_flag     object 
 9   PULocationID           int64  
 10  DOLocationID           int64  
 11  payment_type           int64  
 12  fare_amount            float64
 13  extra                  float64
 14  mta_tax                float64
 15  tip_amount             float64
 16  tolls_amount           float64
 17  improvement_surcharge  float64
 18  total_amount           float64
 19  congestion_surcharge   float64
 20  airport_fee            float64
dtypes: float64(10), int64(8), object(3)
memory usage: 652.

In [12]:
df_pd.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1933507 entries, 0 to 1933506
Data columns (total 21 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Unnamed: 0.1           int64  
 1   Unnamed: 0             int64  
 2   VendorID               int64  
 3   tpep_pickup_datetime   object 
 4   tpep_dropoff_datetime  object 
 5   passenger_count        int64  
 6   trip_distance          float64
 7   RatecodeID             int64  
 8   store_and_fwd_flag     object 
 9   PULocationID           int64  
 10  DOLocationID           int64  
 11  payment_type           int64  
 12  fare_amount            float64
 13  extra                  float64
 14  mta_tax                float64
 15  tip_amount             float64
 16  tolls_amount           float64
 17  improvement_surcharge  float64
 18  total_amount           float64
 19  congestion_surcharge   float64
 20  airport_fee            float64
dtypes: float64(10), int64(8), object(3)
memory usage: 652.

### Selecting Columns

In [13]:
%timeit df_pd[["congestion_surcharge", "airport_fee"]]

5.26 ms ± 81.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%timeit df_fd[["congestion_surcharge", "airport_fee"]]

1.61 ms ± 27 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


### Filtering

In [15]:
%timeit df_pd[df_pd.trip_distance>4]

82.5 ms ± 713 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%timeit df_fd[df_fd.trip_distance>4]

99.5 ms ± 2.81 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Grouping

In [17]:
%timeit df_pd.groupby("PULocationID").mta_tax.mean().reset_index()

28.6 ms ± 634 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%timeit df_fd.groupby("PULocationID").mta_tax.mean().reset_index()

14.3 ms ± 257 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Sorting

In [19]:
%timeit df_pd.sort_values("fare_amount")

1.14 s ± 72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit df_fd.sort_values("fare_amount")

599 ms ± 44.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Concatenating

In [21]:
%timeit pd.concat([df_pd for i in range(10)])

2.07 s ± 13.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit fpd.concat([df_fd for i in range(10)])

16.8 ms ± 1.61 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Merging

In [23]:
%timeit a = pd.merge(df_pd[::100], df_pd[1::100], on = "PULocationID")

3.21 s ± 20.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit a = fpd.merge(df_fd[::100], df_fd[1::100], on = "PULocationID")

8.12 s ± 1.11 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
